In [66]:
import numpy as np
from numba import cuda, float32
import math
print(cuda.gpus)


<Managed Device 0>


In [35]:
cuda.gpus.current

In [36]:
@cuda.jit
def matmul(A, B, C):
    """Perform square matrix multiplication of C = A * B
    """
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        C[i, j] = tmp

In [37]:
N = 2000

A = np.random.random((N,N))
B = np.random.random((N,N))
C = np.zeros((N,N))

In [38]:
%timeit matmul(A, B, C)

CudaAPIError: [719] cuModuleLoadDataEx error:


In [30]:
print(cuda.gpus)

<Managed Device 0>


In [31]:
%timeit A@B

122 ms ± 8.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [32]:
# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 16

@cuda.jit
def fast_matmul(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(bpg):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

In [33]:
%timeit fast_matmul(A, B, C)

43.5 ms ± 4.65 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [22]:
import numpy as np

from numba import guvectorize,vectorize

@guvectorize(['float64[:](float64[:], intp[:], float64[:])'], '(n),()->(n)', target='cuda')
def move_mean(a, window_arr, out):
    window_width = window_arr[0]
    asum = 0.0
    count = 0
    for i in range(window_width):
        asum += a[i]
        count += 1
        out[i] = asum / count
    for i in range(window_width, len(a)):
        asum += a[i] - a[i - window_width]
        out[i] = asum / count



TypingError: Failed at nopython (nopython frontend)
No conversion from none to array(float64, 1d, A) for '$126.2', defined at None
File "<ipython-input-22-266046b1fd32>", line 16
[1] During: typing of assignment at <ipython-input-22-266046b1fd32> (16)

In [21]:
N = 10000
arr = np.arange(N, dtype=np.float64).reshape(2, int(N/2))
%timeit move_mean(arr, 3)

6.87 ms ± 36.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
from __future__ import print_function

from timeit import default_timer as time

import numpy as np
import math
from numba import cuda, jit,prange


@jit('(f4[:], f4[:], f4[:])', parallel=True)
def numba_sum(a, b, c):
    N = len(a)
    for i in prange(N):
        c[i] = np.cos(np.sin(a[i] + b[i]))
        
    
@cuda.jit('(f4[:], f4[:], f4[:])')
def cuda_sum(a, b, c):
    #print(cuda.gridDim.x)
    i = cuda.grid(1)
    c[i] = math.cos(math.sin(a[i] + b[i]))
    
    n = cuda.gridDim.x

griddim = 4096, 1
blockdim = 256, 1, 1
N = griddim[0] * blockdim[0]
print("N", N)
cuda_sum_configured = cuda_sum.configure(griddim, blockdim)
a = np.array(np.random.random(N), dtype=np.float32)
b = np.array(np.random.random(N), dtype=np.float32)
c = np.empty_like(a)
n = 0
ts = time()
cuda_sum_configured(a, b, c)
te = time()
print(ts)



N 1048576
7211.019594523


In [16]:
cuda_sum_configured.device

<weakproxy at 0x7f244deda138 to Device at 0x7f2449bec550>

In [9]:
%timeit numba_sum(a, b, c)
%timeit cuda_sum_configured(a, b, c)

5.71 ms ± 62.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
6.32 ms ± 49.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [27]:
ts

957.994194478

489 µs ± 7.34 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
